ASSIGNMENT 1

Colab Python 예제에 있는 두 가지 문제를 수정하기 

1) 헤더의 값이 들어가는 이슈 제거 

2) 현재 잡을 수행할 때마다 테이블에 레코드들이 들어가는데 이를 idempotent하게 수정하기


In [ ]:
# 1. 헤더의 값이 들어가는 이슈 제거
def transform(text):
    lines = text.split("\n")

    # lines의 첫 element는 column 이름이므로 제외시켜준다.    
    # return lines
    return lines[1:]

In [ ]:
# 2. 현재 잡을 수행할 때마다 테이블에 레코드들이 들어가는데 이를 idempotent하게 수정하기

# load함수가 idempotent하려면 동일한 data를 한 번 입력했을 때 return되는 결과와 여러번 입력했을 때 return되는 결과가 같아야한다.
# 따라서 insert하기 전에 table에 입력되는 data가 이미 존재할 경우 해당 data를 삭제하고 새로 입력한다. 만약 입력되는 데이터 전체가 이미 table에 존재한다면 truncate가 더 효율적이다.

def load(text):
    cur = get_Redshift_connection()
    # 입력되는 데이터 전체가 이미 table에 존재할 경우
    sql = "BEGIN; TRUNCATE TABLE hankyoul0919.name_gender; END;"

    cur.execute(sql);
    for r in text:
        if r != '':
            (name, gender) = r.split(",")
            
            # 입력되는 데이터 전체가 이미 table에 존재할 경우
            sql = "BEGIN; INSERT INTO hankyoul0919.name_gender VALUES ('{name}', '{gender}'); END;".format(name=name, gender=gender)
            
            # 입력되는 데이터 일부만 table에 존재할 경우
            # sql = "BEGIN; DELETE FROM hankyoul0919.name_gender WHERE (name={name} and gender={gender}); INSERT INTO hankyoul0919.name_gender VALUES ('{name}', '{gender}'); END;".format(name=name, gender=gender)
            
            cur.execute(sql)

ASSIGNMENT2

Colab Python 예제를 Airflow의 DAG 포맷으로 바꿔보기. 

* extract, transform, load를 각기 task로 만들지말고 하나의 task안에서 이 세가지를 다 호출.



In [ ]:
import psycopg2

# Redshift connection 함수
def get_Redshift_connection():
    host = "grepp-data.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com"
    redshift_user = "hankyoul0919"
    redshift_pass = "Hankyoul09191!"
    port = 5439
    dbname = "dev"
    conn = psycopg2.connect("dbname={dbname} user={user} host={host} password={password} port={port}".format(
        dbname=dbname,
        user=redshift_user,
        password=redshift_pass,
        host=host,
        port=port
    ))
    conn.set_session(autocommit=True)
    return conn.cursor()

In [ ]:
import requests

def ETL() :
  link = "https://s3-geospatial.s3-us-west-2.amazonaws.com/name_gender.csv"
  cur = get_Redshift_connection()
  extract = requests.get(link).text
  transform = extract.split('\n')[1:]
  truncate = "BEGIN; TRUNCATE TABLE hankyoul0919.name_gender; END;"
  cur.execute(truncate)
  
  for row in transform :
    if row != '' :
      name, gender = row.split(',')
      load = "BEGIN; INSERT INTO hankyoul0919.name_gender VALUES ('{name}', '{gender}'); END;".format(name=name, gender=gender)
      cur.execute(load)
      # print(cur.statusmessage) --> COMMIT





In [ ]:
ETL()